# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [56]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [62]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [63]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [64]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [65]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [66]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [67]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




**Create table artist_song_length_session_table**

In [68]:
query = "CREATE TABLE IF NOT EXISTS artist_song_length_session_table "
query = query + "(sessionid INT, iteminsession SMALLINT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
    #print("artist_song_length_session_table table has been created!")
except Exception as e:
    print(e)  

**Iterate row by row through the file event_datafile_new.csv**

**Get data (column) value for sessionid, iteminsession, artist, song, length**

**Insert the data row by row into table artist_song_length_session_table**

In [69]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_song_length_session_table (sessionid, iteminsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
     
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

**Task 1: Verify that the artist, song and length data were added successfully to the table artist_song_length_session_table**

*ITEMINSESSION AND SESSIONID are the COMPOSITE KEY.* 

*This is to create a unique value while retrieving the query result*

In [70]:
query = "SELECT artist, song, length FROM artist_song_length_session_table WHERE sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

**Create table artist_song_user_session_table**

In [75]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_session_table "
query = query + "(userid SMALLINT, sessionid INT, iteminsession SMALLINT, firstname TEXT, lastname TEXT, artist TEXT, song TEXT, PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
    session.execute(query)
    #print("artist_song_user_session_table table has been created!")
except Exception as e:
    print(e)                 

**Iterate row by row through the file event_datafile_new.csv**

**Get data (column) value for userid, sessionid, iteminsession, firstname, lastname, artist, song**

**Insert the data row by row into table artist_song_user_session_table**

In [76]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_song_user_session_table (userid, sessionid, iteminsession, firstname, lastname, artist, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[1]), str(line[4]), str(line[0]), str(line[9])))

**Task 2: Verify that the artist, song, and user's fullname are added successfully to the table artist_song_user_session_table**

*USERID AND SESSIONID are explicitly defined as the COMPOSITE KEY.* 

*ITEMINSESSION is defined as the CLUSTERING COLUMN to fulfill the sorting technology*

In [77]:
query = "SELECT artist, song, firstname, lastname FROM artist_song_user_session_table WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


**Create table user_song_table**

In [79]:
query = "CREATE TABLE IF NOT EXISTS user_song_table "
query = query + "(song TEXT, userid INT, firstname TEXT, lastname TEXT, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
    #print("user_song_table table has been created!")
except Exception as e:
    print(e) 

**Iterate row by row through the file event_datafile_new.csv**

**Get data (column) value for song, userid, lastname, firstname**

**Insert the data row by row into table user_song_table**

In [80]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_song_table (song, userid, lastname, firstname)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

**Task 3: Verify that the use's full name are added successfully to the table user_song_table**

*SONG AND USERID are the COMPOSITE KEY.* 

*This is to create a unique value while retrieving the query result*

In [81]:
query = "SELECT firstname, lastname FROM user_song_table WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Lynch Jacqueline
Levine Tegan
Johnson Sara


### Drop the tables before closing out the sessions

In [21]:
query = "DROP TABLE artist_song_length_session_table"
try:
    session.execute(query)
    #print("artist_song_length_session_table has been deleted")
except Exception as e:
    print(e)
    
    
query = "DROP TABLE artist_song_user_session_table"
try:
    session.execute(query)
    #print("artist_song_user_session_table has been deleted")
except Exception as e:
    print(e)
    
query = "DROP TABLE user_song_table"
try:
    session.execute(query)
    #print("user_song_table has been deleted")
except Exception as e:
    print(e)
    
    


artist_song_length_session_table has been deleted
artist_song_user_session_table has been deleted
user_song_table has been deleted


### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()